# Depandancies

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
import datetime as dt
from IPython.display import Image
from IPython.core.display import HTML
import sqlite3
from networkx.algorithms import community
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output
from graphviz import Digraph
import graphviz
import pydot
import qrcode
import csv
import os
import sys
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

try:
    import tkinter.ttk
    from tkinter import *
#     from tkinter.ttk import *
except:
    from tkinter import *

from tkinter import filedialog    
from tkinter import messagebox    
from tabulate import tabulate    

In [ ]:
global sqlite_server,e_class
sqlite_server='C:/Users/Crystal/Desktop/Programs/graph_theory/my_gdb.db'
e_class='CLASSIFICATION'
database_table='my_programming'

# Classes

## graph data class

In [ ]:
 class Graph_Data():
    """Instance Source--Target--Attribute data to be graphed"""
    
    headers=['SOURCE','TARGET','DESCR',e_class]
    e_class='TBD'
    type_file='TBD'
    df=pd.DataFrame()
    
    def __init__(self,type_file,e_class):
        
        self.type_file=type_file
        self.e_class=e_class
        Graph_Data.e_class=e_class
        Graph_Data.type_file=type_file
        
        if (self.type_file=='sql'):
            get_data(self.type_file)
            return None
        elif (self.type_file=='csv'):
            filename=input('Enter name of csv file: ')
            self.filename=f'{filename}.csv'
            try:
                self.num_attributes=int(input('num_attributes:'))
            except ValueError:
                self.num_attributes=0
        
            self.headers=['SOURCE', 'TARGET']
        

    def set_attributes(self):
        """Request attribute titles (names) and creates the attibute title list"""
        
        attribute_list=[]
        for i in range(self.num_attributes):
            attribute=input(f'Attribute{i}:')
            attribute_list.append(attribute)
        print(attribute_list)
        self.headers.extend(attribute_list)
    
    def create_file(self):
        """Initially create the file and place file headers"""
        
        if (self.num_attributes>0):
            self.set_attributes()
        
        with open(self.filename, mode='w+',newline='') as test_file:
            test_writer = csv.writer(test_file,dialect='excel', delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            test_writer.writerow(self.headers)
        
        self.num_attributes=0    
        
    def manual_data_entry(self,data_type='csv'):
        """Line by line entry of Source--Target--Attribute data in a csv file """
        
        # Read the first (header)row to get the column count from len(headers)
        with open(self.filename, newline='') as f:
            reader = csv.reader(f)
            self.headers=list(reader)[0]
         
            print(f'before extend: {self.headers}')
            
        if (self.num_attributes>0):
            self.set_attributes()
            print(f'after extend: {self.headers}')
            
            with open(self.filename, 'r+') as csvfile:
                fieldnames = self.headers
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                csvfile.close()
    
            self.num_attributes=0  
        
        row_list=[]
        for i in range(len(self.headers)):
            k=input(f'{self.headers[i]}:')  # 9-21-2019:temporary (optional) interface. Replace with GUI interface.
            row_list.append(k)
        with open(self.filename, mode='a',newline='') as test_file:
            test_writer = csv.writer(test_file,dialect='excel', delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            test_writer.writerow(row_list)
    
        self.data_display()
   
    @classmethod
    def sql_manual_data_entry(cls,row_list,data_type='sql'):
        """Line by line entry of Source--Target--Attribute data in a SQL database table"""
        
#         headers=['SOURCE','TARGET','DESCR','EQUIPMENT_CLASSIFICATION']
        
#         row_list=[]
#         for i in range(len(self.headers)-1):
#             k=input(f'{self.headers[i]}:')  # 9-21-2019:temporary (optional) interface. Replace with GUI interface.
#             row_list.append(k)
        print(Graph_Data.e_class)
        row_list.append(Graph_Data.e_class)
        
        columns=tuple(Graph_Data.headers)
        values=tuple(row_list)
        the_sql_command=f'INSERT INTO {database_table} {columns} VALUES {values}'
        
        
        try:
            conn=sqlite3.connect(sqlite_server)
            cur=conn.cursor()
        except:
            print('UNABLE TO ACCESS DATABASE. TRY AGAIN.')
         
        print(the_sql_command)
        cur.execute(the_sql_command)
        conn.commit()
        
        get_data(Graph_Data.type_file)
        
        
    def revise_data_entry(self):
        """Used to revise, update or correct previous entries"""
        pass
    
    def data_display(self,data_type):
        """Display the data following each entry"""
        path=self.filename
        df1=pd.read_csv(path)
        print(tabulate(df1, headers='keys', tablefmt='psql')) # 9-21-2019:temporary (optional) display. Replace with GUI display
        display(HTML(df1.to_html())) # 9-21-2019:temporary (optional) display. Replace with GUI display
    
    @staticmethod
    def get_graph_data_plot():
        """Call functions to get the Graphviz display of the data"""
        create_edge_list(Graph_Data.df)
    
    
    def external_data_entry(self):
        """Provide data from external file i.e, csv, dataframe, excel,etc."""
        pass
        
        
        


In [ ]:
class Make_Graph():
    """create an instance of a networkx graph for display and analysis"""
    pass

## the interface class

### The Window

In [ ]:
class My_Window(Tk):
    
    now=dt.date.today().strftime('%B %d, %Y')
    time_of_day=dt.datetime.today().strftime('%I:%M:%S %p')

    def __init__(self,parent,*args,**kargs):
        Tk.__init__(self,parent,*args,**kargs)
        self.parent=parent
        self.initialize()
        
        
        
        self.banner=Label(self,text=f'EMR ROOT CAUSE ANALYSIS',fg='white',bg='blue',font='Ariel 30 bold')
        self.banner.grid(row=0,column=0)
        self.date_banner=Label(self,text=f'{My_Window.now}',fg='white',bg='blue',font='Ariel 20 bold')
        self.date_banner.grid(row=1,column=0)
        
    def initialize(self):
        self.title('ROOT CAUSE ANALYSIS')
        self['borderwidth']=4
        self['bg']='blue'
        self.menubar=Menu(self)
        self.menubar.add_command(label="Exit",font='ariel',command=self.bye_bye)
        self.config(menu=self.menubar)
        
    def bye_bye(self):
        self.destroy()
        
        
    


### The Frame

In [ ]:
class The_Frame(Frame):
    
    def __init__ (self,the_window,*args,**kwargs):
        """Instance of the frame"""
        Frame.__init__(self,the_window,*args,**kwargs)
    
        self.frame=Frame(self)
        self.the_window=the_window
        self['background']='purple'
        self['relief']='raised'
        self['borderwidth']=5
        self.grid(row=1,column=0)
        banner_text='Nodes and Attributes'
        self.frame_banner=Label(self,text=banner_text,fg='yellow',bg='red',font='Ariel 15 bold')
        self.frame_banner.grid(row=0,column=0,columnspan=5,pady=15)
        
        self.combo_drop_downs()
        self.submit_button()
        self.file_dialog_button()
        self.display_graph_button()
        self.display_table()
    
    @staticmethod
    def get_drop_down_lists():
        """Obtain the distinct drop down list for SOURCE, TARGET,DESCR and EQUIPMENT CLASSIFICATION from the database."""
        
        
        conn=sqlite3.connect(sqlite_server)
        cur=conn.cursor()
        
        headers=['SOURCE','TARGET','DESCR']
        d1=d2=d3=None
        drop_down_list=[d1,d2,d3]
        for i,k in enumerate (headers):
            the_sql_command=f'select DISTINCT {k} from {database_table} ORDER by {k};'
            print(the_sql_command)
            df=pd.read_sql_query(sql=the_sql_command,con=conn, index_col=None)
            drop_down_list[i]=list(df[k])
        return drop_down_list

    @staticmethod
    def make_a_distict_list(a,b):
        """Utility function to make a distinct list from combined list."""
        c=a+b
        c.sort()
        p=[]
        p.insert(0,c[0])
        print(p)
        for i in range(len(c)):
            if (i>0):
                if (c[i]!=c[i-1]):
                    p.insert(i,c[i])
        p.sort()
        print(p)
        return p
    
    def combo_drop_downs(self):
        """Create combo boxes"""
        drop_down_list=The_Frame.get_drop_down_lists()
        a=drop_down_list[0]   #source list
        b=drop_down_list[1]   #target list
        self.distinct_combo_list=The_Frame.make_a_distict_list(a,b)
        
        
        self.combo_win=Frame(master=self)
        self.source_label=Label(self.combo_win,text=f'SOURCE',fg='white',bg='red',font='Ariel 10 bold')
        self.source_label.pack()
        
        self.source_var=StringVar()
        self.source_combo=ttk.Combobox(self.combo_win,font='Ariel 12 bold',width=30,
                                    background='red',textvariable=self.source_var,values=self.distinct_combo_list)
        self.source_combo.pack()
        
        self.target_label=Label(self.combo_win,text=f'TARGET',fg='white',bg='red',font='Ariel 10 bold')
        self.target_label.pack()
        self.target_var=StringVar()
        self.target_combo=ttk.Combobox(self.combo_win,font='Ariel 12 bold',width=30,
                                    background='blue',textvariable=self.target_var,values=drop_down_list[1])
        self.target_combo.pack()
        
        self.descr_label=Label(self.combo_win,text=f'DESCR',fg='white',bg='red',font='Ariel 10 bold')
        self.descr_label.pack()
        self.descr_var=StringVar()
        self.descr_combo=ttk.Combobox(self.combo_win,font='Ariel 12 bold',width=30,
                                    background='red',textvariable=self.descr_var,values=drop_down_list[2])
        self.descr_combo.pack()
        
        self.combo_win.grid(row=1,column=0)
        
    
    def refresh_display(self):
        """Clear and refresh comboboxes and treeview following an update"""
        self.my_tree.destroy()
        self.display_table()
        
        self.bfs_node_combo.destroy()
        self.the_graph_button.destroy()
        self.display_graph_button()
        
        self.source_combo.destroy()
        self.target_combo.destroy()
        self.descr_combo.destroy()
        self.combo_drop_downs()
        
        self.the_button.destroy()
        self.submit_button()
        
        
    
    def sta_values(self):
        """get the source-target-attribute values from the comboboxes"""
        sta_list=['s','t','a']
        sta_list[0]=self.source_var.get()
        sta_list[1]=self.target_var.get()
        sta_list[2]=self.descr_var.get()
        print(f'stat values:{sta_list}')
        
        Graph_Data.sql_manual_data_entry(sta_list)
        
        The_Frame.get_drop_down_lists()
    
        self.refresh_display()
     
    def submit_button(self):
        """Press button to commit upload to emr_rca table"""
        self.the_button=Button(master=self.combo_win,text='Commit',bg='black',fg='white',relief='raised',command=self.sta_values)
        self.the_button.pack()
    
    
    def file_dialog_button(self):
        """Press button to activate the File Classification class"""
        self.file_dialog_button=Button(master=self,text='File Dialog',bg='black',fg='white',
                                       relief='raised',command=File_Classification)
        self.file_dialog_button.grid(row=2,column=0,sticky=W)
    
    
    
    def display_graph_button(self):
        """Press button for Graphviz display of the data"""
        
        drop_down_list=The_Frame.get_drop_down_lists()
        self.graph_button_win=Frame(master=self)
        self.bfs_node_label=Label(self.graph_button_win,text=f'BFS Start Node',fg='yellow',bg='red',font='Ariel 10 bold')
        self.bfs_node_label.pack()
        
        self.bfs_node_var=StringVar()
        self.bfs_node_combo=ttk.Combobox(self.graph_button_win,font='Ariel 12 bold',width=20,
                                    background='red',textvariable=self.bfs_node_var,
                                         values=self.distinct_combo_list)
        
        self.bfs_node_combo.pack()
        
        self.the_graph_button=Button(master=self.graph_button_win,text='Graph',bg='black',fg='white',relief='raised',
                               command=Graph_Data.get_graph_data_plot)
        self.the_graph_button.pack()
        
        self.graph_button_win.grid(row=1,column=1,padx=20,sticky=None)
    
    
    def display_table(self):
        """Display emr_rca table data"""
        table_display_frame=Frame(master=self)
        
        head_cols=tuple(Graph_Data.df.columns.tolist())
        the_index=Graph_Data.df.index.tolist()
        
        self.my_tree=ttk.Treeview(table_display_frame, height=10,columns=head_cols)
        for col_num,i in enumerate(head_cols,0):
            self.my_tree.heading(str(col_num), text=i,anchor='center')
        
        for j,k in enumerate (the_index,0):
            self.my_tree.insert('',str(j),'index'+str(j),text=str(the_index[j]))

            for m,n in enumerate(head_cols,0):
                self.my_tree.set(item='index'+str(j),column=n,value=Graph_Data.df[n].iloc[j])

        self.my_tree.grid(row=0,column=0,sticky=W)
        yscrollbar = ttk.Scrollbar(table_display_frame, orient='vertical', command=self.my_tree.yview)
        yscrollbar.grid(row=0,column=1,sticky='ns')
        
        self.my_tree.configure(yscrollcommand = yscrollbar.set, selectmode="browse")
        
        
#         self.table_text_box=Text(master=table_display_frame,borderwidth=2,height=20,width=160,font='Ariel 6 bold')
#         the_table=tabulate(Graph_Data.df, headers='keys', tablefmt='github')
#         self.table_text_box.insert(INSERT,the_table)
#         self.table_text_box.pack()
        
        table_display_frame.grid(row=4,column=0,pady=20,sticky=None)
        
    def update_the_database_button(self):
        """Press button to update the database with user changes"""
        
        # get_table_selection_values(self):
#             """get values from doubleclicking the treeview and populate the combo widgets"""
           
        
        #call the Update_Table() class with the selected table values as arguments.
        
#         self.refresh_display()
        pass

## file classification

In [ ]:
class File_Classification():
    """Assign a file,selected in the file dialog box,to a specific classification based on the file type or user preference."""
    
    def __init__(self):
        self.select_file()
    
    def select_file(self):
        my_path='C:\\Users\\Crystal\\Desktop'
        x=filedialog.askopenfilename(initialdir = my_path,title = "Data Repository",filetypes = (("all files","*.*"),("jpeg files","*.jpg"))) 
        filename, file_extension = os.path.splitext(x)
        filesize=round(os.path.getsize(filename+file_extension)/1000)

        self.the_filename=filename.split('/')[-1]
        self.the_path=filename.partition('/'+self.the_filename)[0]
        self.the_extension=file_extension.split('.')[-1]
        
        print(self.the_filename)
        print(self.the_path)
        print(self.the_extension)
        
        self.classify_file_per_extension()
        
    def classify_file_per_extension(self):
        """Assign file classification per the file extension"""
        
        if(self.the_extension=='pdf'):
            self.the_file_classification='Document'
        elif(self.the_extension=='py'):
            self.the_file_classification='Python Program'
        elif(self.the_extension=='ipynb'):
            self.the_file_classification='Python Program Jupyter Notebook'
        elif(self.the_extension=='xlsm'):
            self.the_file_classification='VB Excel'
        elif(self.the_extension=='pptx'):
            self.the_file_classification='Power Point'
        elif(self.the_extension=='csv'):
            self.the_file_classification='Dataset'
        elif(self.the_extension=='db'):
            self.the_file_classification='Database'
            
        print(f'file class:{self.the_file_classification}')
        
        self.send_to_target_descr()
            
    def send_to_target_descr(self):
        """Send the filename and its classification to the comboboxes"""
        the_gui.target_combo.set(f'{self.the_filename}.{self.the_extension}')
        the_gui.descr_combo.set(self.the_file_classification)
        

## Revise database

In [ ]:
class Update_Table():
    """Update the database_table with changes user made following the treeview doubleclick"""
    pass
    

# Functions

In [ ]:
def get_data(type_file,filename=None):
    """read the data (csv file) or (sql table) following an update or upon request"""
    
    if (type_file=='csv'):
        df=pd.read_csv(filename)
        display(HTML(df.to_html()))

        answer=input('View the graph?')
        if (answer=='y'):
            create_edge_list(df)
    elif (type_file=='sql'):
        
        try:
            conn=sqlite3.connect(sqlite_server)
            cur=conn.cursor()
        except:
            print('UNABLE TO ACCESS DATABASE. TRY AGAIN.')
        
        the_sql_command=f'SELECT * FROM {database_table};'
        df=pd.read_sql_query(sql=the_sql_command,con=conn,index_col=None)
        print(tabulate(df, headers='keys', tablefmt='psql'))
        
        display(HTML(df.to_html()))
    
    Graph_Data.df=df
#     view_data=input('View Graph ? ')
#     if (view_data=='y'):
#         create_edge_list(df)
    
    

In [ ]:
def create_edge_list(df):
    """Create the list of edges from the dataset"""
    e = zip(df['SOURCE'],df['TARGET'])
    e=list(e)
    
    print(e)
    
    header_list=df.columns.tolist()
    print(header_list)
    
    create_graph(e,header_list,df)

In [ ]:
def create_graph(e,header_list,df):
    """Create an instance of a networkx graph and map the edges to the attributes"""
    global G
    G = nx.DiGraph()
    G.add_edges_from(e) # creating graph from the edge list
    
    # map edges to attributes
    for k in header_list[2:]: 
        for i,m in enumerate(e):
            G[e[i][0]][e[i][1]][k]=df[k].iloc[i]
            
    BFS()

In [ ]:
def BFS():
    """Conduct Breath First Search (BFS) for the selected node with selected attributes as edge labels"""

#     source_node=input('Enter source node: ')
    source_node=the_gui.bfs_node_var.get()
    if (source_node==""):
        source_node=list(G.edges)[0][0]
    
    k='DESCR'
    
    bfs_path=list(nx.bfs_tree(G, source=source_node, reverse=False, depth_limit=None))
    print(f'The BFS Path: {bfs_path}')

    H=G.subgraph(bfs_path)
    #     list(H)
    edge_label = nx.get_edge_attributes(H, k)
    
    print(f'Edge Label Values: {edge_label.values()}')

    x=edge_label.keys()
    print(f'Edge Label Keys: {x}')
    
    print(f'bfs path[0]: {bfs_path[0]}')
#     return edge_label,bfs_path[0]

    
        
    make_graph(edge_label,source_node)
    

In [ ]:
def make_graph(edge_labels,selected_node,*args):
    """Create a Grapviz plot of the data"""
    
    print(edge_labels)

    graph_set={'rankdir':'TB',
              'landscape':'False',
              'size':'20,16',
              'splines':'polyline'}

    node_set={'color':'blue',
             'shape':'ellipse',
             'style':'filled',
              'fillcolor':'None'}

    edge_set={'headlabel':None,
             'taillabel':None,
             'arrowsize':'1',
             'label':None,
             'labelfontsize':'6',
             'labeldistance':'2.0',
             'labelangle':'45',
             'headport':'c',
             'fontsize':'8',
             'fontcolor':'red'}
    
    g = Digraph(name='learn_1', comment='What goes here', filename=None, directory=None, format=None,
                engine=None, encoding='utf-8', graph_attr=graph_set, 
                node_attr=node_set, edge_attr=edge_set, body=None, strict=False)
    
    
    # Extracting the edge values for labeling the graph edges
    z=list(edge_labels)
    for i in range(len(z)):
        g.edge(z[i][0],z[i][1],label=str(edge_labels.get((z[i][0],z[i][1]))))
    

    g.node(z[0][0], shape='box') #for a BFS, z[0][0] is the first (start) node of the flow
#     g.node(z[i][1], shape='Msquare')  #for a BFS, z[len of the edge list][1] is the last (stop) node of the flow
    
    now=dt.datetime.now()
    tag=now.strftime('%S')
    
    g.view(f'C:/Users/Crystal/Desktop/sidebars/flow_folder/{selected_node}-{tag}')

# Main

In [ ]:
root=My_Window(None)


In [ ]:
# path='C:/Users/Crystal/Desktop/sidebars/flow_folder/'
path='C:/Users/96015/Desktop/Programming/Off_Project_Folders/flow_data/'
name='usd_code'
path+name

In [ ]:
file_type='sql'


In [ ]:
j=Graph_Data(file_type,e_class)

In [ ]:
the_gui=The_Frame(root)

In [ ]:
mainloop()

# Extras